In [1]:
import numpy as np
from sys import exit
from functools import reduce
import sympy
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt

In [42]:
def legendre_symbol(a, n):
        assert(n > a > 0 and n%2 == 1)
        t = 1
        while a != 0:
            while a % 2 == 0:
                a /= 2
                r = n % 8
                if r == 3 or r == 5:
                    t = -t
            a, n = n, a
            if a % 4 == n % 4 == 3:
                t = -t
            a %= n
        if n == 1:
            return t
        else:
            return 0

def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod
 
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

def stringToAscii(s):
    return [ord(c) for c in s]

def stringToBinary(s):
    return [bin(ord(c)) for c in s]

In [50]:
msg = "Máté"
key = 580
print(msg)
asciiList = stringToAscii(msg)
binaryList = stringToBinary(msg)
print(binaryList)
k = ""
for j in binaryList:
    k+=j[2:]
print(list(k))
squareFreeCores = []
for j in range(key,key+len(k)):
    factors = sympy.primefactors(j)
    squareFreeFactorization = []
    for l in factors: 
        helper = j
        count = 0
        while helper%l==0:
            helper/=l
            count+=1
        if count%2==1:
            squareFreeFactorization.append(l)
    squareFreeCores+=squareFreeFactorization
primes = list(set(squareFreeCores))
primes.sort()
print(primes)
a=len(primes)
b=len(k)
print(a,b)
A = np.zeros((b,a))
for x in range(key,key+b):
    for y in primes:
        z = x
        cnt = 0
        while z % y == 0:
            z/=y
            cnt+=1
        if cnt%2 == 1:  
            A[x-key,primes.index(y)]=int(1)
print(A)
print("Solution exists",np.linalg.det(A)) ## We need a non-zero determinant!
msgBin = []
for x in k:
    msgBin.append(int(x))
print(msgBin)
m = np.array(msgBin)
print(m)
A_inv = np.linalg.inv(A) ## for some reason, this seems to be super slow
print(A_inv)
solution = A_inv.dot(m)
print(solution)
print(np.linalg.solve(A,m))
## Once the solution vector for the residues of the primes is established, then we just need to apply
## the Chinese Remainder Theorem for getting the final prime modulus (see paper)


Máté
['0b1001101', '0b11100001', '0b1110100', '0b11101001']
['1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '1']
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 43, 53, 59, 67, 73, 83, 97, 101, 149, 151, 197, 199, 293, 587, 593, 599, 601, 607]
30 30
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 

In [51]:
for i in range(len(solution)):
    solution[i]=int(solution[i])%2
print(solution)
moduli = [1]
for i in range(1,len(primes)):
    for j in range(1,primes[i]):
        if solution[i]==1 and legendre_symbol(j,primes[i])==-1:
            moduli.append(j)
            break
        if solution[i]==0 and legendre_symbol(j,primes[i])==1:
            moduli.append(j)
            break
print(primes)
print(moduli)
print(len(primes),len(moduli))
modulus=chinese_remainder(primes,moduli)
print(modulus)
## let's suppose that p = 1 mod 4

[1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
 0. 1. 0. 1. 0. 0.]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 43, 53, 59, 67, 73, 83, 97, 101, 149, 151, 197, 199, 293, 587, 593, 599, 601, 607]
[1, 2, 1, 3, 2, 1, 1, 2, 5, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 3, 2, 1, 1, 2, 1, 7, 1, 1]
30 30
7419833483365953441636195213120205307487092525147381


In [52]:
realBits = []
for r in range(key,key+len(msgBin)):
    realBits.append(legendre_symbol(r,modulus)%2)
print("Bug is fixed:",realBits==msgBin)


Bug is fixed: False
